## 决策树算法

决策树常用于分类问题, 也是最常用的数据挖掘算法.

+ 邮件分类系统

`K`近邻算法可以完成多分类问题, 但是最大的缺点是无法给出数据的内在含义, 决策树的主要优势就在于数据形式非常容易理解.

![](./figures/tree.png)

### 决策树的构造


**思路:**


+ 从一堆原始数据中构造决策树, 首先我们讨论构造决策树的方法, 编写构造解决树的代码.
+ 度量算法成功率的方法.
+ 使用递归建立分类器, 并使用 `Matplotlib` 绘制决策树.
+ 输入一些隐性数据, 使用决策树分类器进行数据分类.

### 决策树的一般流程

+ 收集数据:可以使用任何方法
+ 准备数据: 树构造算法只使用于标称型数据, 因此数值型数据必须离散化.
+ 分析数据:可以使用任何方法, 构造树完成之后, 我们应该检查图形是否符合预期.
+ 训练算法: 构造树的数据结构.
+ 训练算法: 使用经验计算错误率.
+ 使用算法: 此步骤可以适用于任何监督学习算法, 而使用决策树可以更好地理解数据的内在含义.

### 定义

分类决策树模型是一种描述对实例进行分类的树形结构。决策树由结点`（node）`和有向边`（directed edge）`组成。

结点有两种类型：

内部结点`（internal node）`：表示一个特征或属性。
叶结点`（leaf： node）`：表示一个类。
用决策树分类，从根节点开始，对实例的某一特征进行测试，根据测试结果，将实例分配到其子结点；这时，每一个子结点对应着该特征的一个取值。如此递归地对实例进行测试并分配，直至达到叶结点。最后将实例分配到叶结点的类中。

### 信息增益

在划分数据集之前之后发生的变化称为信息增益.关键在于如何计算信息增益.计算每一个特征值划分数据集获得的信息增益, 获得信息增益的最高特征就是最好的选择.

熵定义为信息的期望, 如果待分类的事物可能划分多个类, 用符号 $x_i$ 的信息定义为

$$
l(x_i) = -log_2p(x_i)
$$

$p(x_i)$ 是选择该分类的概率.

信息熵的计算公式:

$$
Ent(D) = -\sum_{k=1}^{|y|}p_klog_sp_k
$$

$Ent(D)$ 的值越小, 则 $D$ 的纯度越高.

信息增益的计算:


$$
Gain(D,a) = Ent(D) -\sum_{v=1}^{V}\frac{D^v}{D}Ent(D^v)
$$

选择信息增益最大的于是它被选为划分的属性.对给个分之节点进行上述的计算.最終得到决策树.

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import operator

In [20]:
# 计算给定数据集的信息熵

def clacShannonENt(dataSet):
    numEntrie = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannonEnt = 0.0
    for key in labelCounts:
        prob = np.float(labelCounts[key])/numEntrie
        shannonEnt -= prob*np.log2(prob)
    return shannonEnt

创建数据集进行测试

In [3]:
def creatDataSet():
    dataSet = [[1,1,'yes'],
              [1,1,'yes'],
              [1,0,'no'],
              [0,1,'no'],
              [0,1,'no']]
    label = ['no surfacing','flippers']
    return dataSet,label

In [4]:
dataSet,label = creatDataSet()
shannonEnt = clacShannonENt(dataSet)
shannonEnt

0.97095059445466858

熵越高,混合的数据越多.我们可以在数据集中添加更多的分类, 观察熵是如何变化的. 

In [6]:
dataSet[0][-1]='maybe'

In [7]:
shannonEnt = clacShannonENt(dataSet)
shannonEnt

1.3709505944546687

接下来我们学习如何划分数据集以及如何度量信息熵.

其中 `C4.5` 决策树算法, 不直接使用信息增益, 使用的是增益率.来选择最优划分属性,计算公式为

$$
Gain Ratio(D,a) = \frac{Gain(D,a)}{IV(a)}
$$

其中 

$$
IV(a) = -\sum_{v=1}^{V}log_2\frac{D^v}{D}
$$

成为属性 a 的固有值, 属性 a 的可能取值数目越越多(V越大) 则,IV(a)　通常越大.　

增益率的准则对可取数目较少的属性有所偏好.先从候选划分属性中赵出信息增益高于平均水平的属性.再从中选出增益率最高的.

#### 基尼指数

`CART` 决策树使用的基尼指数来选择划分属性,　数据集(D)的纯度可以使用基尼指数来度量

$$
Gini(D) = \sum_{k=1}^{|y|}\sum_{k'\neq k}p^kp_k' =1 - \sum_{k=1}^{|y|}p_k^2
$$

其中 Gini(D) 反映从数据集 D 中随机算去两个样本, 其类别标记不一致的概率.

Gini(D) 越小数据集纯度越高.

$$
Gini_index(D,a) = \sum_{v=1}^{V}\frac{|D^v|}{|D|}Gini(D^v)
$$

我们选择划分后最小的基尼指数的属性作为最优的分类标准.

In [23]:
# 按照特定的特征划分数据集.

def splitDataSet(dataSet,axis,value):
    reDataSet = []
    for featVer in dataSet:
        if featVer[axis] == value:
            reducedFeatVec = featVer[:axis]
            reducedFeatVec.extend(featVer[axis+1:])
            reDataSet.append(reducedFeatVec)
    return reDataSet

In [15]:
dataSet

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [19]:
splitDataSet(dataSet,0,1)

[1, 1, 'maybe']
[1, 1, 'yes']
[1, 0, 'no']
[0, 1, 'no']
[0, 1, 'no']


[[1, 'maybe'], [1, 'yes'], [0, 'no']]

In [17]:
splitDataSet(dataSet,0,0)

[[1, 'no'], [1, 'no']]

接下来我们将遍历整个数据集, 循环计算信息熵, 找到最好的特征划分.

In [24]:
# 选择最好的数据集划分方式

def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = clacShannonENt(dataSet)
    bestInforGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntorpy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet,i,value)
            prop = len(subDataSet)/np.float(len(dataSet))
            newEntorpy += prop*clacShannonENt(subDataSet)
        infoGain = baseEntropy - newEntorpy
        if infoGain > bestInforGain:
            bestInforGain = infoGain
            bestFeature = i
    return bestFeature

In [25]:
chooseBestFeatureToSplit(dataSet)

0

### 递归构建决策树

In [27]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        classCount[vote] +=1
    sortedClassCount = sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]

### 创建树函数的代码

In [ ]:
# 创建树函数的代码

def creatTree(dataSet,label):
    classList = [example[i] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = label[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(label[bestFeat])
    
    featValues = [example[bestFeat] for example in dataSet]
    
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = label[:]
        mytree[bestFeatLabel][value] = creatTree(splitDataSet)